## Analysis of Controller on Nonlinear Simulation

This notebook generates plots for figures TODO in the paper TODO.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
from __future__ import annotations

from glob import glob
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

# hide top and right splines on plots
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

In [ ]:
plots_dir = 'plots/tsg_take2/'
os.makedirs(plots_dir, exist_ok=True)

def savefig(fig: plt.Figure, filename: str, **kwargs) -> None:
    path = os.path.join(plots_dir, filename)
    defaults = dict(dpi=300, pad_inches=0, bbox_inches='tight', facecolor='white')
    fig.savefig(path, **(defaults | kwargs))

In [ ]:
TIME_TICKS =  [   0, 2400, 4800,  7200,  9600, 12000, 14400]
TIME_LABELS = ['0h', '4h', '8h', '12h', '16h', '20h', '24h']

v_min, v_max = (11.4**2, 12.6**2)  # +/-5%, units kV^2
print(v_min, v_max)

y_min, y_max = 11.2, 12.8

In [ ]:
# Recreate Fig8 in Qu and Li (2020)
# - they count the substation as bus 1
# - we count the substation as bus 0
buses = [8, 18, 21, 30, 39, 45, 54]  # 0 = substation

In [ ]:
from network_utils import (
    create_56bus,
    create_RX_from_net,
    read_load_data)

net = create_56bus()
R, X = create_RX_from_net(net, noise=0)  # true R and X
p, qe = read_load_data()  # in MW and MVar
T, n = p.shape

v_nom = 12**2  # nominal squared voltage magnitude, units kV^2
v_sub = v_nom  # fixed squared voltage magnitude at substation, units kV^2

# vpars = qe @ X + p @ R + v_sub  # shape [T, n]

# saved nonlinear vpars includes substation. Here, we ignore the substation.
vpars = np.load('data/nonlinear_voltage_baseline.npy')[:, 1:]  # shape [T, n]
vpars = (vpars * 12.)**2
Vpar_min = np.min(vpars, axis=0) - 0.5
Vpar_max = np.max(vpars, axis=0) + 0.5
Vpar = (Vpar_min, Vpar_max)

In [ ]:
print('max-voltage node:', np.argmax(vpars.max(axis=0)))
print('min-voltage node:', np.argmin(vpars.min(axis=0)))

## Figure 2b: nonlinear simulation, no controller

In [ ]:
def plot_2b():
    # plot nonlinear sim no-control
    fig, ax = plt.subplots(figsize=(4, 3), dpi=200, tight_layout=True)

    ts = range(T)
    for i in np.asarray(buses) - 1:
        ax.plot(ts, np.sqrt(vpars[:, i]))

    ax.axhline(11.4, ls='--', color='black')
    ax.axhline(12.6, ls='--', color='black')
    ax.set(ylabel='Voltage (kV)', ylim=(11.0, 13.4))
    ax.set(xlabel='time $t$', xlim=(0, T),
           xticks=TIME_TICKS, xticklabels=TIME_LABELS)

    savefig(fig, filename='nonlinear_nocontrol.pdf')
    savefig(fig, filename='nonlinear_nocontrol.png')
    savefig(fig, filename='nonlinear_nocontrol.svg')

In [ ]:
# plot_2b()

## Fig1 - Baselines
- voltage profile, no controller
- voltage profile, controller with true X
- voltage profile, Li et al. controller I
- voltage profile, Li et al. controller II

In [ ]:
def load_pkl(path: str) -> dict:
    with open(path, 'rb') as f:
        return pickle.load(f)


outdir = 'out/nonlinear/'
pkl_paths = {
    # ('known', None): outdir + 'CBCconst_20230810_130611.pkl',  # fixed X̂, fixed etahat
    ('known', None): outdir + 'CBCconst_δ20_η10_20230810_130842.pkl',  # fixed X̂, learned etahat
}
for seed in [8, 9, 10, 11]:
    pkl_paths |= {
        # default: δ=20
        ('unknown', seed): glob(outdir + f'CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_2*.pkl')[0],
        ('topo-14', seed): glob(outdir + f'CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knowntopo14_2*.pkl')[0],
        ('lines-14', seed): glob(outdir + f'CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],

        (r'η* known', seed): glob(outdir + f'CBCproj_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=1', seed): glob(outdir + f'CBCproj_δ1_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=20', seed): glob(outdir + f'CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=100', seed): glob(outdir + f'CBCproj_δ100_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
        (r'δ=500', seed): glob(outdir + f'CBCproj_δ500_η10_noise1.0_perm_norm1.0_seed{seed}_knownlines14_2*.pkl')[0],
    }

pkls = {}
for (name, seed), pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[(name, seed)] = pkl
    print(f'{name: <15} {str(seed): <4}', pkl.keys())

In [ ]:
def check_consistency(data: dict):
    v = data['vs']
    qc = data['qcs']
    u = qc[1:] - qc[:-1]  # u[t] = u(t) = q^c(t+1) - q^c(t)
    Δv = v[1:] - v[:-1]  # Δv[t] = v(t+1) - v(t)
    count_inconsistent = []
    for t in sorted(data['params'].keys()):
        if data['config']['δ'] == 0:
            X̂ = data['params'][t]
            etahat = 8.65
        else:
            X̂, etahat = data['params'][t]
        vpar_hat = v[1:t+1] - qc[1:t+1] @ X̂
        w_hat = Δv[:t] - u[:t] @ X̂
        consistent = (
            (Vpar_min - 0.05 <= vpar_hat).all(axis=1)
            | (vpar_hat <= Vpar_max + 0.05).all(axis=1)
            | (np.max(np.abs(w_hat), axis=1) <= etahat)
        )
        num_inconsistent = t - consistent.sum()
        count_inconsistent.append(num_inconsistent)
    return count_inconsistent

In [ ]:
count_inconsistent = {}
for (name, seed), data in tqdm(pkls.items()):
    count_inconsistent[(name, seed)] = check_consistency(data)

In [ ]:
def plot_pkl(name: str, data: dict, seed: int | None = None, plot_legend: bool = False) -> None:
    ts = range(T)
    fig, ax = plt.subplots(figsize=(4, 3), dpi=60, tight_layout=True)
    for i in np.array(buses) - 1:
        ax.plot(ts, np.sqrt(data['vs'][:, i]), label=f'{i+1}')

    ax.axhline(11.4, ls='--', color='black')
    ax.axhline(12.6, ls='--', color='black')
    ax.set(ylabel='Voltage (kV)', ylim=(y_min, y_max),
           yticks=[11.4, 11.7, 12, 12.3, 12.6])
    ax.set(xlabel='time $t$', xlim=(-50, T),
           xticks=TIME_TICKS, xticklabels=TIME_LABELS)

    filename = f'nonlinear_{name}'
    if seed is not None:
        filename += f'_s{seed}'
    savefig(fig, filename=f'{filename}.pdf')
    savefig(fig, filename=f'{filename}.png')
    savefig(fig, filename=f'{filename}.svg')

    if plot_legend:
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='bus')
        fig.canvas.draw()
        bbox = leg.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        savefig(fig, 'nonlinear_legend.pdf', bbox_inches=bbox)
        savefig(fig, 'nonlinear_legend.png', bbox_inches=bbox)
        savefig(fig, 'nonlinear_legend.svg', bbox_inches=bbox)

In [ ]:
def fig3abcd(seeds: list[int]) -> None:
    for seed in seeds:
        for i, name in enumerate(['unknown', 'topo-14', 'lines-14']):
            print(name, seed)
            data = pkls[(name, seed)]
            plot_pkl(name, data, seed, plot_legend=(i == 0))
    plot_pkl('known', pkls[('known', None)], None)


fig3abcd(seeds=[8, 9, 10, 11])

In [ ]:
def plot_error_and_etahat(pkls_dict: dict[str, dict], filename: str,
                          legend: str | None) -> None:
    """
    Args
    - legend: one of [None, 'top', 'separate']
    """
    fig, ax = plt.subplots(figsize=(4, 3), dpi=60, tight_layout=True)
    axr = ax.twinx()
    axr.spines['right'].set_visible(True)

    for name, data in pkls_dict.items():
        print(name)
        ax.step(data['dists']['t'] + [T], data['dists']['X_true'] + [data['dists']['X_true'][-1]],
                where='post', label=name)
        ax.scatter(0, data['dists']['X_true'][0])
        if 'η' in data['dists']:
            axr.step([0] + data['dists']['t'] + [T], [0] + data['dists']['η'] + [data['dists']['η'][-1]], ':',
                     where='post')
        else:
            axr.plot([0, T], [8.65, 8.65], ':')

    ax.set_ylabel(r'$||\hat{X}_t - X^\star||_\bigtriangleup$')
    axr.set_ylabel(r'$\hat\eta$')
    ax.set(xticks=TIME_TICKS, xticklabels=TIME_LABELS)
    ax.set(xlabel='time $t$', xlim=(-50, T))

    if legend == 'top':
        ax.legend(ncols=2, bbox_to_anchor=(0, 1), loc='lower left')

    savefig(fig, filename=f'{filename}.pdf')
    savefig(fig, filename=f'{filename}.png')
    savefig(fig, filename=f'{filename}.svg')

    if legend == 'separate':
        axr.set_ylabel('')
        axr.set_yticklabels([])
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.canvas.draw()
        bbox = leg.get_tightbbox().transformed(fig.dpi_scale_trans.inverted())
        savefig(fig, f'{filename}_legend.pdf', bbox_inches=bbox)
        savefig(fig, f'{filename}_legend.png', bbox_inches=bbox)
        savefig(fig, f'{filename}_legend.svg', bbox_inches=bbox)

In [ ]:
def fig3e(seeds: list[int]) -> None:
    for seed in seeds:
        fig3e_pkls = {
            name: pkls[(name, seed)]
            for name in ['unknown', 'topo-14', 'lines-14']
        }
        fig3e_pkls['known'] = pkls[('known', None)]
        plot_error_and_etahat(fig3e_pkls, filename=f'nonlinear_error_s{seed}', legend='top')


fig3e(seeds=[8, 9, 10, 11])

In [ ]:
def fig5a(seeds: list[int]) -> None:
    for i, seed in enumerate(seeds):
        pkls_by_delta = {
            name: pkls[(name, seed)]
            for name in ['η* known', 'δ=1', 'δ=20', 'δ=100', 'δ=500']
        }
        plot_error_and_etahat(
            pkls_by_delta, filename=f'nonlinear_error_by_delta_s{seed}',
            legend='separate' if i == 0 else None)


fig5a(seeds=[8, 9, 10, 11])

## Fig6 - Partial Control

In [ ]:
pkl_paths = {
    ('known', None): outdir + 'CBCconst_δ20_η10_partialctrl_20230811_194616.pkl',  # fixed X̂, learned etahat
}
for seed in [8, 9, 10, 11]:
    pkl_paths |= {
        # default: δ=20
        ('unknown', seed): glob(outdir + f'CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_partialctrl_2*.pkl')[0],
        ('lines-14', seed): glob(outdir + f'CBCproj_δ20_η10_noise1.0_perm_norm1.0_seed{seed}_partialctrl_knownlines14_2*.pkl')[0],
    }

pkls = {}
for (name, seed), pkl_path in pkl_paths.items():
    pkl = load_pkl(pkl_path)
    pkls[(name, seed)] = pkl
    print(f'{name: <15} {str(seed): <4}', pkl.keys())

In [ ]:
def plot_fill(ax: plt.Axes, values: np.ndarray, color: int, label: str, alpha=False) -> None:
    """
    Args
    - values: shape [num_runs, T]
    - color: int, index into tab20 colors
        0 = blue, 2 = orange, 4 = green, 7 = purple
    """
    num_runs, T = values.shape
    ts = range(T)
    dark = plt.cm.tab20.colors[color]
    light = plt.cm.tab20.colors[color + 1]

    if num_runs == 1:
        ax.plot(ts, values[0], color=dark, lw=0.5, label=label)
    else:
        mean = values.mean(axis=0)
        std = values.std(axis=0)
        ax.plot(ts, mean, color=dark, lw=0.5, label=label)
        if alpha:
            ax.fill_between(ts, mean-std, mean+std, color=light, alpha=0.5)
        else:
            ax.fill_between(ts, mean-std, mean+std, color=light)

In [ ]:
def plot_bus(pkls_by_label: dict, bus: int, legend: bool = False) -> None:
    """
    Args:
    - bus: int, where bus 0 = substation
    """
    fig, ax = plt.subplots(figsize=(4, 3), tight_layout=True)
    for c, (label, pkls) in enumerate(pkls_by_label.items()):
        num_runs = len(pkls)
        vs = np.zeros([num_runs, T])
        for i, data in enumerate(pkls):
            vs[i] = np.sqrt(data['vs'][:, bus - 1])
        plot_fill(ax, vs, color=c*2, label=label, alpha=True)

    ax.axhline(11.4, ls='--', color='black')
    ax.axhline(12.6, ls='--', color='black')
    ax.set(ylabel='Voltage (kV)', ylim=(y_min, y_max),
           yticks=[11.4, 11.7, 12, 12.3, 12.6])
    ax.set(xlabel='time $t$', xlim=(-50, T),
           xticks=TIME_TICKS, xticklabels=TIME_LABELS)

    savefig(fig, filename=f'nonlinear_partialctrl_bus{bus}.pdf')
    savefig(fig, filename=f'nonlinear_partialctrl_bus{bus}.png')
    savefig(fig, filename=f'nonlinear_partialctrl_bus{bus}.svg')

    if legend:
        leg = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        fig.canvas.draw()
        bbox = leg.get_tightbbox().transformed(fig.dpi_scale_trans.inverted())
        savefig(fig, 'nonlinear_partialctrl_legend.pdf', bbox_inches=bbox)
        savefig(fig, 'nonlinear_partialctrl_legend.png', bbox_inches=bbox)
        savefig(fig, 'nonlinear_partialctrl_legend.svg', bbox_inches=bbox)

In [ ]:
def fig6(seeds: list[int]) -> None:
    for i, bus in enumerate([18, 30]):
        pkls_by_label = {
            'unknown': [pkls[('unknown', seed)] for seed in seeds],
            'lines-14': [pkls[('lines-14', seed)] for seed in seeds],
            'known': [pkls[('known', None)]],
        }
        plot_bus(pkls_by_label, bus=bus, legend=(i==0))

fig6(seeds=[8, 9, 10, 11])